<a href="https://colab.research.google.com/github/ElleuchMed2022/ML-programming-/blob/main/fine_tuning_camembert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
!pip install SentencePiece

In [3]:
# Importing standard libraries for every machine/deep learning pipeline
import pandas as pd
import torch
from tqdm import tqdm, trange
import numpy as np


# Importing specific libraries for data prerpcessing, model archtecture choice, training and evaluation
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW
# import torch.optim as optim
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import seaborn as sns


In [4]:
from google.colab import files
uploaded = files.upload()

Saving data_4463.xlsx to data_4463.xlsx


In [8]:
import io 
df = pd.read_excel(io.BytesIO(uploaded["data_4463.xlsx"]))
df.head(5)

,Unnamed: 0,Poste,Entreprise,Contrat,description,Localisation,salaire €
0,0,Tuyauteur (H/F),ARTUS INTERIM VANNES,Intérim,Notre agence ARTUS INTERIM de Vannes recherche...,\n ARTUS INTERI...,5000-25000
1,1,MECANICIEN PL (H/F),SOC GARAGE VEHICULES INDUSTRIELS,CDI,En tant que Mécanicien PL et rattaché au Chef ...,Évreux,25000-50000
2,2,Opérateur de production (H/F),Manpower SABLE SUR SARTHE,Intérim 3 mois,Manpower SABLE SUR SARTHE recherche pour son c...,NaN,5000-25000
3,3,TECHNICIEN ELECTROMECA SAV H/F,DE GRAËT CONSULTING|DE GRAET CONSULTING,CDI,Dans le cadre du développement constant de son...,NaN,25000-50000
4,4,ELECTRICIEN H/F,MENCO SAINT NAZAIRE,Intérim,"""MENCO ST NAZAIRE recrute des Électriciens con...",\n FINANCIERE M...,5000-25000


In [9]:
del df["Unnamed: 0"]

In [10]:
df.head(5)

,Poste,Entreprise,Contrat,description,Localisation,salaire €
0,Tuyauteur (H/F),ARTUS INTERIM VANNES,Intérim,Notre agence ARTUS INTERIM de Vannes recherche...,\n ARTUS INTERI...,5000-25000
1,MECANICIEN PL (H/F),SOC GARAGE VEHICULES INDUSTRIELS,CDI,En tant que Mécanicien PL et rattaché au Chef ...,Évreux,25000-50000
2,Opérateur de production (H/F),Manpower SABLE SUR SARTHE,Intérim 3 mois,Manpower SABLE SUR SARTHE recherche pour son c...,NaN,5000-25000
3,TECHNICIEN ELECTROMECA SAV H/F,DE GRAËT CONSULTING|DE GRAET CONSULTING,CDI,Dans le cadre du développement constant de son...,NaN,25000-50000
4,ELECTRICIEN H/F,MENCO SAINT NAZAIRE,Intérim,"""MENCO ST NAZAIRE recrute des Électriciens con...",\n FINANCIERE M...,5000-25000


In [11]:
# Defining constants
epochs = 4
MAX_LEN = 192
batch_size =16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [12]:
# Initialize CamemBERT tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

In [13]:
# Creates list of texts and labels
text=[]
text0 = df['Poste'].values.tolist()
text1 = df['description'].values.tolist()
text2 = df['Entreprise'].values.tolist()
labels = df['salaire €'].values.tolist()

for i in  range(df.shape[0]):
  m=text0[i]+" "+text1[i]
  text.append(m)



In [14]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
labels = LE.fit_transform(labels)

In [15]:
labels

array([1, 0, 1, ..., 2, 2, 0])

In [16]:
#user tokenizer to convert sentences into tokenizer
input_ids  = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in text]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [17]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [18]:
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)

In [19]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,
                                                            random_state=42, test_size=0.1)

In [20]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [21]:
# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [22]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=3)
model.to(device)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

In [23]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [24]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [25]:
# Store our loss and accuracy for plotting if we want to visualize training evolution per epochs after the training process
train_loss_set = []

In [26]:
for _ in trange(epochs, desc="Epoch"):  
    # Tracking variables for training
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the model
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # Get loss value
        loss = outputs[0]
        # Add it to train loss list
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
    
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    


    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Validation of the model
    model.eval()
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs[:2]
    
        # Move logits and labels to CPU if GPU is used
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.6721371218740227


Epoch:  25%|██▌       | 1/4 [04:30<13:31, 270.51s/it]

Validation Accuracy: 0.8229166666666667
Train loss: 0.3485546161038467


Epoch:  50%|█████     | 2/4 [09:00<09:00, 270.19s/it]

Validation Accuracy: 0.8745535714285715
Train loss: 0.21761501157485633


Epoch:  75%|███████▌  | 3/4 [13:30<04:30, 270.19s/it]

Validation Accuracy: 0.8834821428571429
Train loss: 0.13590515691386276


Epoch: 100%|██████████| 4/4 [18:00<00:00, 270.21s/it]

Validation Accuracy: 0.9037202380952382


In [ ]:
desc = ["En tant que Mécanicien PL et rattaché au Chef d'Atelier, vos différentes missions seront de :- Diagnostiquer et réaliser l'ensemble des prestations techniques nécessaires à la maintenance des Poids-Lourds Industriels et VUL qui vous sont confiés dans le respect des méthodes de travail et des règles de sécurité ;- Etablir des diagnostiques de pannes via les valises et établir les réparations ;- Gérer professionnellement vos documents d'intervention (fiche diagnostic, ordre de réparation…) ;- Veiller au rangement et à la propreté de l'atelier ;- Détecter les éléments à changer (Usure, danger…) ;- Préparer les véhicules pour le passage aux mines ;- Utiliser les bancs de freinage ;- Effectuer des interventions pour les dépannages sur routes ;- Intervenir sur les éléments suivants : moteurs, boîtes de vitesses, pont, suspension, mise à niveau, graissage, vidange, système de freinage, limitateur de vitesse, hayon élévateur…Sur une base de 37h00 par semaine les horaires sont :- Du Lundi au Vendredi de 8h00-12h00 14h00-17h30.- Samedi 08h00-12h00 (1 Samedi sur 2)."]

In [ ]:
# Encode the comments
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in desc]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)